In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json

In [ ]:
print(os.listdir('../own_dataset/2d6_zoom/train'))

In [ ]:
train_path= ('../own_dataset/2d6_zoom/train')
test_path= ('../own_dataset/2d6_zoom/test')

In [ ]:
batch_size_train=30  #,batch_size= batch_size_train
batch_size_valid=30
targetsize= 150
datagen=ImageDataGenerator(rotation_range=360,brightness_range=[0.7,1.2],height_shift_range=0.1
                           ,width_shift_range=0.1,zoom_range=[0.7,1.1])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), 
                                           classes=os.listdir('../own_dataset/2d6_zoom/train'),
                                           batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), 
                                           classes=os.listdir('../own_dataset/2d6_zoom/test'),
                                           batch_size= batch_size_train)

In [ ]:
def plots(ims, figsize=(20,10), rows=1, interp= False, titles= None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims= ims.transpose((0,1,2,3))
    f= plt.figure(figsize=figsize)
    cols= len(ims)//rows if len(ims) %2 == 0 else len(ims)//rows + 1
    for i in range(5):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
plots(train_batches[1][0],figsize=(100,70))

In [ ]:

model2 = Sequential()
model2.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))


model2.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())
model2.add(Dense(6, activation='softmax'))
model2.summary()

In [ ]:
model2.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model2.fit_generator(train_batches, steps_per_epoch= len(train_batches) , callbacks=[reduce_lr],
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=50)

In [ ]:
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=os.getcwd(),save_weights_only=True,verbose=1)

In [ ]:
model2.save('model_2d6_zoom2.h5')

In [ ]:
model2_json = model2.to_json()
with open("model_2d6_zoom2.json", "w") as json_file:
    json_file.write(model2_json)